In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_finance_self_service.a_daily_store_mch2_data 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
as
select
    a.calday as day,
    a.store_id,
    a.mch2_id,
    coalesce(a.billcnt_excl_sll, 0) as billcount
FROM
    {catalog_name}.udp_wcm_gold_vcm_dmt.a_billcnt_mch2 a
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON a.STORE_ID = b.STORE_ID
WHERE
    (a.CALDAY BETWEEN '2021-12-01' AND date_sub(CURRENT_DATE(), 1)) AND
    b.BUSINESS_UNIT IN ('1500', '2000', '6000')
""")